# Construccion de triangulo y creación de funciones para data_tarea3

In [4]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

# =======================================
# DATOS DE ENTRADA DESDE CHUNK 2
# =======================================
INPUT_PATH = "data_tarea3.xlsx"
RATIO_ELR_GLOBAL = 0.8  # ELR único (global)
print("Cargando datos y construyendo triángulo...")

# --- Funciones necesarias del Chunk 2 para generar el triángulo ---
def load_data(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"No se encontró el archivo: {path}")
    df = pd.read_excel(path)
    df = df.rename(columns=lambda c: str(c).strip())
    for c in df.columns:
        lc = str(c).lower()
        if 'evento' in lc or 'ocur' in lc:
            df = df.rename(columns={c: 'Fecha Evento'})
        if 'notifi' in lc or 'f_pago' in lc or 'not' in lc:
            df = df.rename(columns={c: 'Fecha Notifi'})
        if 'monto' in lc or 'siniest' in lc or 'pago' in lc:
            df = df.rename(columns={c: 'Monto Siniestro'})
    if not set(['Fecha Evento', 'Fecha Notifi', 'Monto Siniestro']).issubset(df.columns):
        raise ValueError(f"Columnas esperadas no encontradas. Columnas actuales: {df.columns.tolist()}")
    df['Fecha Evento'] = pd.to_datetime(df['Fecha Evento'], errors='coerce')
    df['Fecha Notifi'] = pd.to_datetime(df['Fecha Notifi'], errors='coerce')
    df = df.dropna(subset=['Fecha Evento', 'Fecha Notifi'])
    df['Monto Siniestro'] = pd.to_numeric(df['Monto Siniestro'], errors='coerce').fillna(0.0)
    return df

def build_triangle_accumulated(df):
    df = df.copy()
    df['AccidentYear'] = df['Fecha Evento'].dt.year
    df['DevLag'] = (df['Fecha Notifi'].dt.year - df['AccidentYear']).astype(int)
    df = df[df['DevLag'] >= 0]
    tri_inc = df.pivot_table(index='AccidentYear', columns='DevLag',
                             values='Monto Siniestro', aggfunc='sum', fill_value=0.0)
    tri_inc = tri_inc.sort_index(axis=0).sort_index(axis=1)
    maxlag = int(tri_inc.columns.max())
    tri_inc = tri_inc.reindex(columns=range(maxlag+1), fill_value=0.0)
    triangle_acum = tri_inc.cumsum(axis=1)
    latest_year = triangle_acum.index.max()
    for ay in triangle_acum.index:
        years_available = latest_year - ay
        if years_available + 1 < triangle_acum.shape[1]:
            triangle_acum.loc[ay, years_available+1:] = np.nan
    return triangle_acum

# --- Cargar datos y construir triángulo ---
df = load_data(INPUT_PATH)
triangle = build_triangle_accumulated(df)

# --- Definir primas desde el chunk 2 ---
primas = pd.Series({
    2005: 26426000,
    2006: 34611000,
    2007: 40045000,
    2008: 44158000,
    2009: 48994000,
    2010: 52421000,
    2011: 56597000,
    2012: 59675000,
    2013: 59342000
})

# --- Crear serie de ELR (constante 0.8 para todos los años) ---
elr = pd.Series(RATIO_ELR_GLOBAL, index=triangle.index)

print("\nTriángulo acumulado listo para análisis:")
print(triangle)

# =======================================
# FUNCIONES ORIGINALES (NO MODIFICADAS)
# =======================================
def calc_linkratios_and_cdf(tri):
    lags = list(tri.columns)
    f = []
    for k in range(len(lags)-1):
        col_k = lags[k]
        col_k1 = lags[k+1]
        mask = tri[[col_k, col_k1]].notna().all(axis=1)
        num = tri.loc[mask, col_k1].sum()
        den = tri.loc[mask, col_k].sum()
        if den == 0:
            raise ZeroDivisionError(f"Denominador cero para f_{k} (lag {col_k}). Revisa el triángulo y los NaN.")
        fk = num/den
        f.append(fk)
    f = np.array(f, dtype=float)
    CDF = np.ones(len(f)+1, dtype=float)
    for i in range(len(f)-1, -1, -1):
        CDF[i] = CDF[i+1] * f[i]
    cdf_series = pd.Series(CDF, index=lags)
    if (cdf_series < 1 - 1e-12).any():
        print("WARNING: alguna CDF < 1. Revisa link ratios. Valores:\n", cdf_series)
    return f, cdf_series

def metodo_chain_ladder(tri):
    f, CDF = calc_linkratios_and_cdf(tri)
    rows = []
    for ay in tri.index:
        row = tri.loc[ay]
        if row.notna().sum() == 0:
            raise ValueError(f"AY {ay} sin observaciones.")
        edad = int(row.last_valid_index())
        C_obs = float(row.loc[edad])
        cdf_val = float(CDF.loc[edad])
        U_CL = C_obs * cdf_val
        IBNR_CL = U_CL - C_obs
        rows.append([ay, C_obs, cdf_val, U_CL, IBNR_CL])
    df = pd.DataFrame(rows, columns=["AY","C_obs","CDF","U_CL","IBNR_CL"]).set_index("AY")
    return df, CDF

def metodo_bornhuetter_ferguson(tri, primas, elr, CDF):
    rows = []
    for ay in tri.index:
        row = tri.loc[ay]
        edad = int(row.last_valid_index())
        cdf_val = float(CDF.loc[edad])
        p_i = 1.0 / cdf_val
        uno_menos_p = 1.0 - p_i
        IBNR_BF = float(elr.loc[ay]) * float(primas.loc[ay]) * uno_menos_p
        rows.append([ay, float(elr.loc[ay]), float(primas.loc[ay]), p_i, uno_menos_p, IBNR_BF])
    df = pd.DataFrame(rows, columns=["AY","ELR_i","Prima","p_i","1-p_i","IBNR_BF"]).set_index("AY")
    return df

def metodo_cape_cod(tri, primas, CDF):
    rows = []
    num_sum = 0.0
    den_sum = 0.0
    for ay in tri.index:
        row = tri.loc[ay]
        edad = int(row.last_valid_index())
        C_obs = float(row.loc[edad])
        cdf_val = float(CDF.loc[edad])
        p_i = 1.0 / cdf_val
        uno_menos_p = 1.0 - p_i
        num_sum += C_obs
        den_sum += float(primas.loc[ay]) * p_i
        rows.append([ay, C_obs, float(primas.loc[ay]), p_i, uno_menos_p])
    theta_hat = num_sum / den_sum
    rows2 = []
    for r in rows:
        ay, C_obs, prima, p_i, uno_menos_p = r
        IBNR_CC = theta_hat * prima * uno_menos_p
        rows2.append([ay, C_obs, prima, p_i, uno_menos_p, IBNR_CC])
    df = pd.DataFrame(rows2, columns=["AY","C_obs","Prima","p_i","1-p_i","IBNR_CC"]).set_index("AY")
    df["theta_hat"] = theta_hat
    return df, theta_hat

# =======================================
# EJECUCIÓN DE LOS MÉTODOS
# =======================================
df_cl, CDF = metodo_chain_ladder(triangle)
df_bf = metodo_bornhuetter_ferguson(triangle, primas, elr, CDF)
df_cc, theta_hat = metodo_cape_cod(triangle, primas, CDF)

# =======================================
# RESUMEN FINAL
# =======================================
resumen = pd.DataFrame({
    "Método": ["Chain Ladder", "Bornhuetter–Ferguson", "Cape Cod"],
    "IBNR Total": [df_cl["IBNR_CL"].sum(), df_bf["IBNR_BF"].sum(), df_cc["IBNR_CC"].sum()]
})

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
print("\n--- MÉTODO CHAIN LADDER ---")
print(df_cl)
print(f"Total IBNR_CL = {df_cl['IBNR_CL'].sum():,.2f}")

print("\n--- MÉTODO BORNHUETTER–FERGUSON ---")
print(df_bf)
print(f"Total IBNR_BF = {df_bf['IBNR_BF'].sum():,.2f}")

print("\n--- MÉTODO CAPE COD ---")
print(df_cc)
print(f"Theta_hat = {theta_hat:.9f}")
print(f"Total IBNR_CC = {df_cc['IBNR_CC'].sum():,.2f}")

print("\n=== RESUMEN FINAL ===")
print(resumen)


Cargando datos y construyendo triángulo...

Triángulo acumulado listo para análisis:
DevLag                  0            1             2             3  \
AccidentYear                                                         
2005           367,000.00 3,167,000.00  4,928,000.00  6,156,000.00   
2006         1,025,000.00 4,246,000.00  5,639,000.00  8,209,000.00   
2007         1,007,000.00 4,535,000.00  7,346,000.00  9,367,000.00   
2008         1,587,000.00 7,104,000.00 10,020,000.00 12,664,000.00   
2009         2,206,000.00 8,444,000.00 11,202,400.00 13,851,900.00   
2010         2,194,000.00 6,748,000.00 10,491,000.00 14,780,900.00   
2011         1,165,000.00 6,145,800.00 12,758,100.00           NaN   
2012         1,064,600.00 8,172,400.00           NaN           NaN   
2013         1,402,400.00          NaN           NaN           NaN   

DevLag                   4             5             6             7  \
AccidentYear                                                           


Al comparar los resultados obtenidos mediante los tres métodos de estimación de reservas, se observa que el Chain Ladder arroja un IBNR total de aproximadamente 78,86 millones, reflejando directamente los patrones históricos de desarrollo de siniestros. Este método es adecuado cuando la experiencia pasada es estable, pero puede llegar a subestimar la reserva en los años recientes donde la maduración aún es parcial. Por su parte, el Bornhuetter–Ferguson genera la reserva más alta, alrededor de 143,13 millones, al combinar la información histórica con la expectativa a priori basada en el ELR (que en este caso fue ELR=0.8) y las primas. Esto lo hace más prudente y conservador, especialmente útil en situaciones de alta incertidumbre o cambios recientes en la siniestralidad. El Cape Cod, con un IBNR de aproximadamente 79,42 millones, estima el ELR internamente a partir de los datos observados, ofreciendo un equilibrio entre la experiencia histórica y la expectativa teórica. Considerando que el triángulo muestra patrones consistentes y los años recientes presentan desarrollo parcial, se recomienda adoptar el método Cape Cod como base para la reserva técnica, ya que proporciona un valor razonable, evita sobre-reservar como puede ser que se este haciendo en BF dado al alto valor del ratio de siniestralidad dado, además permite justificar la consistencia entre los datos observados y la exposición futura.

## BORRADOR: Prueba de que estén bien calculados usando Ej 1 Misc

In [5]:
import pandas as pd
import numpy as np

# --------------------------
# Datos del ejercicio (triángulo y primas / ELR)
# --------------------------
triangle_data = {
    0: [9502, 8138, 9802, 9498, 9072],
    1: [25827, 26292, 25563, 25266, np.nan],
    2: [37275, 37496, 37257, np.nan, np.nan],
    3: [44083, 42114, np.nan, np.nan, np.nan],
    4: [44490, np.nan, np.nan, np.nan, np.nan]
}
triangle = pd.DataFrame(triangle_data, index=[0, 1, 2, 3, 4])
triangle.index.name = "AY"
triangle.columns.name = "DevLag"

primas = pd.Series([52600, 54000, 55520, 57500, 59850], index=[0, 1, 2, 3, 4])
elr = pd.Series([0.84, 0.82, 0.80, 0.80, 0.80], index=[0, 1, 2, 3, 4])

# --------------------------
# Probar los métodos con la información original
# --------------------------
df_cl, CDF = metodo_chain_ladder(triangle)
df_bf = metodo_bornhuetter_ferguson(triangle, primas, elr, CDF)
df_cc, theta_hat = metodo_cape_cod(triangle, primas, CDF)

# --------------------------
# Resumen de resultados
# --------------------------
resumen = pd.DataFrame({
    "Método": ["Chain Ladder", "Bornhuetter–Ferguson", "Cape Cod"],
    "IBNR Total": [df_cl["IBNR_CL"].sum(), df_bf["IBNR_BF"].sum(), df_cc["IBNR_CC"].sum()]
})

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print("\n--- MÉTODO CHAIN LADDER ---")
print(df_cl)
print(f"Total IBNR_CL = {df_cl['IBNR_CL'].sum():,.2f}")

print("\n--- MÉTODO BORNHUETTER–FERGUSON ---")
print(df_bf)
print(f"Total IBNR_BF = {df_bf['IBNR_BF'].sum():,.2f}")

print("\n--- MÉTODO CAPE COD ---")
print(df_cc)
print(f"Theta_hat = {theta_hat:.9f}")
print(f"Total IBNR_CC = {df_cc['IBNR_CC'].sum():,.2f}")

print("\n=== RESUMEN FINAL ===")
print(resumen)



--- MÉTODO CHAIN LADDER ---
       C_obs  CDF      U_CL   IBNR_CL
AY                                   
0  44,490.00 1.00 44,490.00      0.00
1  42,114.00 1.01 42,502.82    388.82
2  37,257.00 1.16 43,346.91  6,089.91
3  25,266.00 1.68 42,392.88 17,126.88
4   9,072.00 4.68 42,420.96 33,348.96
Total IBNR_CL = 56,954.56

--- MÉTODO BORNHUETTER–FERGUSON ---
    ELR_i     Prima  p_i  1-p_i   IBNR_BF
AY                                       
0    0.84 52,600.00 1.00   0.00      0.00
1    0.82 54,000.00 0.99   0.01    405.08
2    0.80 55,520.00 0.86   0.14  6,240.11
3    0.80 57,500.00 0.60   0.40 18,584.17
4    0.80 59,850.00 0.21   0.79 37,640.55
Total IBNR_BF = 62,869.90

--- MÉTODO CAPE COD ---
       C_obs     Prima  p_i  1-p_i   IBNR_CC  theta_hat
AY                                                     
0  44,490.00 52,600.00 1.00   0.00      0.00       0.79
1  42,114.00 54,000.00 0.99   0.01    389.01       0.79
2  37,257.00 55,520.00 0.86   0.14  6,142.38       0.79
3  25,266.00 57,5